#**Project Description**

This notebook implements a multi-stage quantitative finance model designed not to achieve maximal portfolio performance, but rather to **compare and contrast the simultaneous behavior of three fundamental portfolio construction methodologies**:


1.  **MVOR (Mean-Variance Optimized Portfolio):** Maximizing the Sharpe Ratio based on Deep Learning-predicted returns.


2.  **GMV (Global Minimum Volatility Portfolio):** Minimizing portfolio risk regardless of returns.


3.  **ERC (Equal Risk Contribution Portfolio):** Distributing risk equally among the assets.

The core objective is to explore the differences in asset allocation dictated by these classic strategies when fed the same set of predictions and risk data.

## **Key Methodological Choices & Research Question**

### 1. **Advanced Input Generation**

* **Asset Universe:** A basket of 18 major cryptocurrencies (e.g., BTC-USD, ETH-USD, SOL-USD).

* **Feature Engineering:** Technical indicators (RSI, MACD, Bollinger Bands, etc.) are computed using the `ta` library to enrich the feature set.

* **Deep Learning Prediction:** A **Stacked LSTM (Long Short-Term Memory) network** is trained on historical price data and technical indicators to forecast the one-period-ahead expected returns ($\mu$) for *all assets simultaneously*.

* **Risk Filtering:** K-Means Clustering on historical volatility is used to select a "**low-risk**" subset of cryptocurrencies for the final optimization step.

### 2. **The Simulation Premise**: Cryptos as Stocks

A central experimental assumption of this project is to treat the 24/7 cryptocurrency market **as if it were a traditional stock market**.

* **Risk Annualization:** The covariance matrix and volatility are annualized using a **252-day factor** (the approximate number of trading days in a year for equity markets), **rather than** the 365-day factor typical for 24/7 markets.

This deliberate choice directly addresses the **following research question**:

> **What are the portfolio allocation results, risk metrics, and strategy divergences if I treat a universe of cryptocurrencies as if they were normal stocks?**

**WARNING**:

This notebook is not a financial advisor.

In [ ]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=b4a68e7b7e02d4cc4da2a27381689cd98c0ef61aa45541da9ff2fafbb30f3a48
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import ta
from scipy.optimize import minimize
import datetime as dt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [ ]:


MAX_WEIGHT_PER_CRYPTO = 0.15
MIN_WEIGHT_PER_CRYPTO = 0.01
MIN_CRYPTO_IN_PORTFOLIO = 8

RISK_FREE_RATE = 0.02


LOOKBACK = 60
EPOCHS = 10
BATCH_SIZE = 32


start_date = "2020-01-01"
end_date = "2024-12-31"


CRYPTO_TICKERS = [
    'BTC-USD',
    'ETH-USD',
    'BNB-USD',
    'SOL-USD',
    'XRP-USD',
    'ADA-USD',
    'AVAX-USD',
    'DOGE-USD',
    'DOT-USD',
    'TRX-USD',
    'LINK-USD',
    'MATIC-USD',
    'LTC-USD',
    'BCH-USD',
    'ATOM-USD',
    'UNI-USD',
    'XLM-USD',
    'ETC-USD',
]

ALL_TICKERS = CRYPTO_TICKERS



def download_crypto_data(tickers, start_date, end_date):

    print(f"Downloading data for {len(tickers)} cryptocurrencies...")

    all_data = {}
    successful_tickers = []

    for ticker in tickers:
        try:
            data = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
            if data.empty:
                print(f"    No data for {ticker}")
                continue


            if not all(col in data.columns for col in ['Open', 'High', 'Low', 'Close', 'Volume']):
                print(f"    Missing required columns for {ticker}")
                continue


            if len(data) < 100:
                print(f"    Insufficient data points for {ticker}")
                continue

            all_data[ticker] = data
            successful_tickers.append(ticker)
            print(f"    Successfully downloaded {ticker}")

        except Exception as e:
            print(f"    Error downloading {ticker}: {str(e)}")

    print(f"Successfully downloaded {len(successful_tickers)} out of {len(tickers)} cryptocurrencies")
    return all_data, successful_tickers

def compute_technical_indicators_single(data_df, ticker):

    try:

        close_prices = data_df['Close'].squeeze()
        high_prices = data_df['High'].squeeze()
        low_prices = data_df['Low'].squeeze()
        open_prices = data_df['Open'].squeeze()
        volume = data_df['Volume'].squeeze()

        indicators = {}


        indicators['RSI'] = ta.momentum.RSIIndicator(close=close_prices, window=14).rsi()


        macd = ta.trend.MACD(close=close_prices)
        indicators['MACD'] = macd.macd()
        indicators['MACD_Signal'] = macd.macd_signal()
        indicators['MACD_Histogram'] = macd.macd_diff()


        indicators['SMA_20'] = ta.trend.SMAIndicator(close=close_prices, window=20).sma_indicator()
        indicators['SMA_50'] = ta.trend.SMAIndicator(close=close_prices, window=50).sma_indicator()
        indicators['EMA_12'] = ta.trend.EMAIndicator(close=close_prices, window=12).ema_indicator()
        indicators['EMA_26'] = ta.trend.EMAIndicator(close=close_prices, window=26).ema_indicator()


        bollinger = ta.volatility.BollingerBands(close=close_prices, window=20, window_dev=2)
        indicators['BB_Upper'] = bollinger.bollinger_hband()
        indicators['BB_Lower'] = bollinger.bollinger_lband()
        indicators['BB_Middle'] = bollinger.bollinger_mavg()
        indicators['BB_Width'] = (indicators['BB_Upper'] - indicators['BB_Lower']) / indicators['BB_Middle']


        indicators['ATR'] = ta.volatility.AverageTrueRange(
            high=high_prices, low=low_prices, close=close_prices, window=14
        ).average_true_range()


        stoch = ta.momentum.StochasticOscillator(high=high_prices, low=low_prices, close=close_prices, window=14)
        indicators['Stoch_K'] = stoch.stoch()
        indicators['Stoch_D'] = stoch.stoch_signal()


        indicators['Volume'] = volume
        indicators['Volume_SMA'] = ta.trend.SMAIndicator(close=volume, window=20).sma_indicator()
        indicators['OBV'] = ta.volume.OnBalanceVolumeIndicator(close=close_prices, volume=volume).on_balance_volume()


        indicators['Price_Change'] = close_prices.pct_change()
        indicators['High_Low_Ratio'] = high_prices / low_prices
        indicators['Close_Open_Ratio'] = close_prices / open_prices

        indicators_df = pd.DataFrame(indicators, index=data_df.index)


        indicators_df = indicators_df.ffill().bfill()

        indicators_df = indicators_df.dropna()

        return indicators_df

    except Exception as e:
        print(f"Error computing indicators for {ticker}: {str(e)}")
        return None

def build_lstm_model(input_shape, output_units):

    inputs = Input(shape=input_shape)
    x = LSTM(units=64, return_sequences=True)(inputs)
    x = Dropout(0.3)(x)
    x = LSTM(units=32, return_sequences=True)(x)
    x = Dropout(0.3)(x)
    x = LSTM(units=16)(x)
    x = Dropout(0.2)(x)
    outputs = Dense(output_units)(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model


def portfolio_return(weights, mu):
    return np.dot(weights, mu)

def portfolio_volatility(weights, Sigma):
    return np.sqrt(np.dot(weights.T, np.dot(Sigma, weights)))

def neg_sharpe_ratio(weights, mu, Sigma, risk_free_rate):
    p_ret = portfolio_return(weights, mu)
    p_vol = portfolio_volatility(weights, Sigma)
    if p_vol < 1e-6:
        return 1e10
    return - (p_ret - risk_free_rate) / p_vol

def objective_min_risk_contrib(weights, Sigma, N_STOCKS):

    p_vol = portfolio_volatility(weights, Sigma)

    if p_vol < 1e-10:
        return 0
    MCR = np.dot(Sigma, weights) / p_vol
    RC = weights * MCR
    target_RC = np.ones_like(weights) / N_STOCKS
    return np.sum((RC - target_RC)**2)

def diversification_penalty(weights, min_crypto=8, threshold=0.005):

    meaningful_crypto = np.sum(weights > threshold)
    if meaningful_crypto < min_crypto:
        return 1000 * (min_crypto - meaningful_crypto)
    return 0



print("--- 1. DATA ACQUISITION & FEATURE ENGINEERING ---")


crypto_data, successful_tickers = download_crypto_data(ALL_TICKERS, start_date, end_date)

if len(successful_tickers) < 10:
    print(" Insufficient cryptocurrencies with valid data. Exiting.")
else:

    print("\nAligning data to common date range...")
    all_closes = pd.DataFrame()
    all_indicators = {}


    for ticker in successful_tickers:
        all_closes[ticker] = crypto_data[ticker]['Close']


        indicators_df = compute_technical_indicators_single(crypto_data[ticker], ticker)
        if indicators_df is not None:
            all_indicators[ticker] = indicators_df


    common_dates = all_closes.index
    for ticker in all_indicators.keys():
        common_dates = common_dates.intersection(all_indicators[ticker].index)

    if len(common_dates) < 100:
        print(" Insufficient common dates after alignment. Exiting.")
    else:
        print(f"Common date range: {common_dates.min()} to {common_dates.max()}")
        print(f"Total trading days: {len(common_dates)}")


        all_closes_aligned = all_closes.loc[common_dates].dropna(axis=1)


        final_tickers = all_closes_aligned.columns.tolist()


        final_indicators = {
            ticker: all_indicators[ticker].loc[common_dates]
            for ticker in final_tickers if ticker in all_indicators
        }


        final_tickers = [t for t in final_tickers if t in final_indicators]
        all_closes_aligned = all_closes_aligned[final_tickers]

        print(f"Final dataset: {all_closes_aligned.shape[1]} cryptocurrencies, {all_closes_aligned.shape[0]} trading days")



        print("\n--- 2. K-MEANS CLUSTERING (Risk Filtering - FIXED) ---")

        daily_returns = all_closes_aligned.pct_change().dropna()

        if daily_returns.empty:
            print(" No valid returns data. Exiting.")
        else:
            annual_volatility = daily_returns.std() * np.sqrt(252)
            volatility_df = pd.DataFrame(annual_volatility, columns=['Annual_Volatility']).dropna()

            if volatility_df.empty:
                print(" No valid volatility data. Exiting.")
            else:
                print(f"Volatility range: {volatility_df['Annual_Volatility'].min():.3f} to {volatility_df['Annual_Volatility'].max():.3f}")

                volatility_matrix = volatility_df.values
                scaler_risk = MinMaxScaler()
                scaled_risk = scaler_risk.fit_transform(volatility_matrix)


                K = min(4, len(volatility_df))


                if K < MIN_CRYPTO_IN_PORTFOLIO:
                     print(f" Only {len(volatility_df)} assets available, which is less than MIN_CRYPTO_IN_PORTFOLIO ({MIN_CRYPTO_IN_PORTFOLIO}). Selecting all available assets.")
                     least_risk_tickers = volatility_df.index.tolist()
                else:
                    kmeans = KMeans(n_clusters=K, random_state=42, n_init='auto')
                    clusters = kmeans.fit_predict(scaled_risk)
                    volatility_df['Cluster'] = clusters

                    cluster_centers = kmeans.cluster_centers_


                    sorted_cluster_indices = np.argsort(cluster_centers.flatten())


                    least_risk_tickers = []

                    for cluster_id in sorted_cluster_indices:
                        current_cluster_tickers = volatility_df[volatility_df['Cluster'] == cluster_id].index.tolist()
                        least_risk_tickers.extend(current_cluster_tickers)

                        if len(least_risk_tickers) >= MIN_CRYPTO_IN_PORTFOLIO:

                            break


                    least_risk_tickers = list(set(least_risk_tickers))


                N_STOCKS = len(least_risk_tickers)

                print(f"Total available cryptocurrencies: {len(final_tickers)}")
                print(f"Selected Low-Risk Cryptocurrencies for Optimization: {N_STOCKS}")
                print(f"Low-Risk Cryptocurrencies: {least_risk_tickers}")



                if N_STOCKS < MIN_CRYPTO_IN_PORTFOLIO:
                    print(f" FINAL ERROR: Despite fix, selected assets ({N_STOCKS}) is less than required minimum ({MIN_CRYPTO_IN_PORTFOLIO}). This should only happen if total available assets < {MIN_CRYPTO_IN_PORTFOLIO}.")
                    print("Optimization will proceed with available assets, but diversification constraints might be impossible.")



                selected_tickers = least_risk_tickers
                data_dl_close = all_closes_aligned[selected_tickers].copy()


                final_indicators_selected = {
                    ticker: final_indicators[ticker] for ticker in selected_tickers
                }



                print("\n--- 3. DEEP LEARNING (LSTM) FOR PREDICTION WITH TECHNICAL INDICATORS ---")


                print("Preparing combined price and technical indicator data...")

                X_combined_list = []
                y_combined_list = []
                price_scalers = {}

                for ticker in selected_tickers:
                    price_data = data_dl_close[[ticker]].copy()
                    indicators_data = final_indicators_selected[ticker]


                    scaler_price = MinMaxScaler(feature_range=(0, 1))
                    price_normalized = scaler_price.fit_transform(price_data)
                    price_scalers[ticker] = scaler_price


                    scaler_indicators = MinMaxScaler(feature_range=(0, 1))
                    indicators_normalized = scaler_indicators.fit_transform(indicators_data)


                    features_normalized = np.concatenate([price_normalized, indicators_normalized], axis=1)


                    X_crypto, y_crypto = [], []
                    for i in range(len(features_normalized) - LOOKBACK):
                        X_crypto.append(features_normalized[i:(i + LOOKBACK)])

                        y_crypto.append(price_normalized[i + LOOKBACK])

                    X_combined_list.append(np.array(X_crypto))
                    y_combined_list.append(np.array(y_crypto))



                min_length = min([len(X) for X in X_combined_list])
                X_aligned = [X[:min_length] for X in X_combined_list]
                y_aligned = [y[:min_length] for y in y_combined_list]


                X_combined = np.stack(X_aligned, axis=2)
                y_combined = np.stack(y_aligned, axis=1).squeeze()


                n_samples, lookback, n_features, n_assets = X_combined.shape
                X_reshaped = X_combined.reshape(n_samples, lookback, n_features * n_assets)

                print(f"Combined training sequences: {X_reshaped.shape}")
                print(f"Target shape: {y_combined.shape}")

                if len(X_reshaped) == 0:
                    print(" Insufficient data for sequence creation. Exiting.")
                else:
                    TRAIN_SIZE = int(0.8 * len(X_reshaped))
                    X_train = X_reshaped[:TRAIN_SIZE]
                    y_train = y_combined[:TRAIN_SIZE]

                    print(f"Training sequences: {X_train.shape}")
                    print(f"Features per sequence: {X_train.shape[2]}")


                    model = build_lstm_model(input_shape=(LOOKBACK, X_train.shape[2]), output_units=N_STOCKS)

                    print("Starting LSTM model training with technical indicators")
                    history = model.fit(
                        X_train,
                        y_train,
                        epochs=EPOCHS,
                        batch_size=min(BATCH_SIZE, len(X_train)),
                        verbose=1,
                        validation_split=0.1
                    )
                    print("Model training complete.")


                    last_lookback_data = X_reshaped[-1:].reshape(1, LOOKBACK, X_reshaped.shape[2])
                    predicted_scaled_prices = model.predict(last_lookback_data, verbose=0)


                    predicted_prices_list = []
                    for i, ticker in enumerate(selected_tickers):
                        scaler_price = price_scalers[ticker]


                        pred_scaled = predicted_scaled_prices[0, i].reshape(-1, 1)
                        pred_price = scaler_price.inverse_transform(pred_scaled)[0, 0]
                        predicted_prices_list.append(pred_price)

                    predicted_prices_series = pd.Series(predicted_prices_list, index=selected_tickers)
                    last_close_prices = data_dl_close.iloc[-1]
                    mu_predictions = (predicted_prices_series / last_close_prices) - 1
                    mu_predictions.name = 'Expected_Return_LSTM'

                    print("\nPredicted returns:")
                    print(mu_predictions.round(4))



                    print("\n--- 4. PORTFOLIO OPTIMIZATION (With STRICT Diversification) ---")


                    returns = data_dl_close.pct_change().dropna()
                    returns = returns[mu_predictions.index]

                    if returns.empty:
                        print(" No valid returns data for optimization. Exiting.")
                    else:
                        Sigma = returns.cov() * 252
                        mu_array = mu_predictions.values
                        Sigma_array = Sigma.values

                        print(f"Covariance matrix shape: {Sigma_array.shape}")


                        constraints = [
                            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
                        ]


                        bounds = tuple((MIN_WEIGHT_PER_CRYPTO, MAX_WEIGHT_PER_CRYPTO) for asset in range(N_STOCKS))
                        initial_weights = np.array([1/N_STOCKS] * N_STOCKS)


                        min_possible_sum = MIN_WEIGHT_PER_CRYPTO * N_STOCKS
                        max_possible_sum = MAX_WEIGHT_PER_CRYPTO * N_STOCKS

                        if max_possible_sum < 1.0:
                            print(f" IMPOSSIBLE CONSTRAINT: Max weight (15%) * {N_STOCKS} assets = {max_possible_sum*100}%. Cannot sum to 100%. Adjust MAX_WEIGHT_PER_CRYPTO.")
                        elif min_possible_sum > 1.0:
                            print(f" IMPOSSIBLE CONSTRAINT: Min weight (1%) * {N_STOCKS} assets = {min_possible_sum*100}%. Cannot sum to 100%. Adjust MIN_WEIGHT_PER_CRYPTO.")
                        else:
                            print("Running optimization strategies with strict diversification")


                            def neg_sharpe_ratio_diversified(weights, mu, Sigma, risk_free_rate):
                                base_sharpe = neg_sharpe_ratio(weights, mu, Sigma, risk_free_rate)
                                div_penalty = diversification_penalty(weights, MIN_CRYPTO_IN_PORTFOLIO)
                                return base_sharpe + div_penalty

                            def portfolio_volatility_diversified(weights, Sigma):
                                base_vol = portfolio_volatility(weights, Sigma)
                                div_penalty = diversification_penalty(weights, MIN_CRYPTO_IN_PORTFOLIO)
                                return base_vol + div_penalty

                            def objective_min_risk_contrib_diversified(weights, Sigma, N_STOCKS):
                                base_obj = objective_min_risk_contrib(weights, Sigma, N_STOCKS)
                                div_penalty = diversification_penalty(weights, MIN_CRYPTO_IN_PORTFOLIO)
                                return base_obj + div_penalty


                            strategies_weights = {}

                            try:

                                opt_mvor = minimize(neg_sharpe_ratio_diversified, initial_weights,
                                                    args=(mu_array, Sigma_array, RISK_FREE_RATE),
                                                    method='SLSQP', bounds=bounds, constraints=constraints)
                                if opt_mvor.success:
                                    strategies_weights['MVOR (Max Sharpe)'] = opt_mvor.x
                                else:
                                    print(f"  MVOR optimization failed: {opt_mvor.message}")


                                opt_gmv = minimize(portfolio_volatility_diversified, initial_weights,
                                                   args=(Sigma_array,),
                                                   method='SLSQP', bounds=bounds, constraints=constraints)
                                if opt_gmv.success:
                                    strategies_weights['GMV (Min Volatility)'] = opt_gmv.x
                                else:
                                    print(f"  GMV optimization failed: {opt_gmv.message}")


                                opt_erc = minimize(objective_min_risk_contrib_diversified, initial_weights,
                                                   args=(Sigma_array, N_STOCKS),
                                                   method='SLSQP', bounds=bounds, constraints=constraints)
                                if opt_erc.success:
                                    strategies_weights['ERC (Equal Risk Contrib.)'] = opt_erc.x
                                else:
                                    print(f"  ERC optimization failed: {opt_erc.message}")

                            except Exception as e:
                                print(f" Optimization error: {str(e)}")

                            if not strategies_weights:
                                print(" All optimizations failed. Exiting.")
                            else:

                                def analyze_portfolio_diversification(weights, tickers, threshold=0.005):

                                    weights_series = pd.Series(weights, index=tickers)
                                    meaningful_allocation = weights_series[weights_series > threshold]
                                    n_crypto = len(meaningful_allocation)

                                    top_weight = weights_series.max()
                                    top_2_weights = weights_series.nlargest(2).sum()
                                    top_5_weights = weights_series.nlargest(5).sum()
                                    herfindahl = np.sum(weights_series ** 2)

                                    return {
                                        'n_crypto_meaningful': n_crypto,
                                        'max_weight': top_weight,
                                        'top_2_concentration': top_2_weights,
                                        'top_5_concentration': top_5_weights,
                                        'herfindahl_index': herfindahl
                                    }

                                def get_portfolio_metrics(weights, mu, Sigma, rf):
                                    weights = np.array(weights)
                                    p_ret = portfolio_return(weights, mu)
                                    p_vol = portfolio_volatility(weights, Sigma)
                                    p_sharpe = (p_ret - rf) / p_vol if p_vol > 1e-6 else 0
                                    return p_ret, p_vol, p_sharpe


                                performance_metrics = {}
                                final_weights = {}
                                diversification_metrics = {}

                                for name, w_array in strategies_weights.items():
                                    w_final = np.array(w_array)
                                    p_ret, p_vol, p_sharpe = get_portfolio_metrics(w_final, mu_array, Sigma_array, RISK_FREE_RATE)

                                    performance_metrics[name] = (p_ret, p_vol, p_sharpe)
                                    final_weights[name] = pd.Series(w_final, index=selected_tickers)
                                    diversification_metrics[name] = analyze_portfolio_diversification(w_final, selected_tickers)



                                print("\n--- 5. RESULTS AND DIVERSIFICATION ANALYSIS ---")


                                metrics_df = pd.DataFrame(performance_metrics,
                                                         index=['Expected Annual Return', 'Annual Volatility', 'Sharpe Ratio']).T

                                print("\n### 5.1 Portfolio Performance Comparison ###")
                                print(metrics_df.round(4).to_markdown(numalign="left", stralign="left"))


                                print(f"\n### 5.2 Diversification Analysis ###")
                                div_df = pd.DataFrame(diversification_metrics).T
                                print(div_df.round(4).to_markdown(numalign="left", stralign="left"))


                                full_weights_df = pd.DataFrame(final_weights).T

                                print("\n### 5.3 Portfolio Weights ###")

                                print(full_weights_df.map(lambda x: f"{x:.2%}").to_markdown(numalign="left", stralign="left"))


                                print(f"\n### 5.4 Concentration Warnings ###")
                                for strategy, div_metrics in diversification_metrics.items():
                                    warnings = []
                                    if div_metrics['max_weight'] > 0.20:
                                        warnings.append(f"Single crypto concentration: {div_metrics['max_weight']:.1%}")
                                    if div_metrics['top_2_concentration'] > 0.35:
                                        warnings.append(f"Top 2 concentration: {div_metrics['top_2_concentration']:.1%}")
                                    if div_metrics['n_crypto_meaningful'] < MIN_CRYPTO_IN_PORTFOLIO:
                                        warnings.append(f"Only {div_metrics['n_crypto_meaningful']} cryptos with meaningful weights")

                                    if warnings:
                                        print(f"{strategy}: {' | '.join(warnings)}")
                                    else:
                                        print(f"{strategy}: Well diversified ")

                                print(f"\nDiversification Constraints Applied:")
                                print(f"- Maximum weight per crypto: {MAX_WEIGHT_PER_CRYPTO:.1%}")
                                print(f"- Minimum weight per crypto: {MIN_WEIGHT_PER_CRYPTO:.1%}\n")

--- 1. DATA ACQUISITION & FEATURE ENGINEERING ---
    Successfully downloaded BTC-USD
    Successfully downloaded ETH-USD
    Successfully downloaded BNB-USD
    Successfully downloaded SOL-USD
    Successfully downloaded XRP-USD
    Successfully downloaded ADA-USD
    Successfully downloaded AVAX-USD
    Successfully downloaded DOGE-USD
    Successfully downloaded DOT-USD
    Successfully downloaded TRX-USD
    Successfully downloaded LINK-USD
    Successfully downloaded MATIC-USD
    Successfully downloaded LTC-USD
    Successfully downloaded BCH-USD
    Successfully downloaded ATOM-USD
    Successfully downloaded UNI-USD
    Successfully downloaded XLM-USD
    Successfully downloaded ETC-USD
Successfully downloaded 18 out of 18 cryptocurrencies

Aligning data to common date range...
Common date range: 2020-09-22 00:00:00 to 2024-12-30 00:00:00
Total trading days: 1561
Final dataset: 18 cryptocurrencies, 1561 trading days

--- 2. K-MEANS CLUSTERING (Risk Filtering - FIXED) ---
Volati

# **Interpreting the Results**

* **Portfolio Constraints:**

All optimization runs are subjected to **strict diversification constraints** (Minimum weight of $1.0\%$ and Maximum weight of $15.0\%$ per asset), ensuring a practical, balanced portfolio, regardless of the optimization strategy.

* **Sharpe Ratio Context:** The low resultant Sharpe Ratios are acknowledged and expected.

This outcome is a direct consequence of combining the LSTM's volatile short-term predictions with a risk metric that is deliberately miscalculated for the crypto market (via the 252-day stock-like simulation).

The performance numbers reflect the difficulty of achieving high risk-adjusted returns under this simulation environment, but this does not detract from the primary goal of **methodological comparison**.

**Warning**:

This notebook is not a financial advisor.